### Exemple Of Pyspark ML

### 1. Linear Regression

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

22/11/19 21:09:33 WARN Utils: Your hostname, dell-5570 resolves to a loopback address: 127.0.1.1; using 10.189.123.35 instead (on interface wlp0s20f3)
22/11/19 21:09:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/19 21:09:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/19 21:09:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/19 21:09:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/19 21:09:34 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/11/19 21:09:34 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
## Read The Dataset
training =  spark.read.csv('data/salary.csv', header = True, inferSchema = True)

In [4]:
training.show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Omar| 31|         3| 30000|
| Hamm| 39|        11|120000|
|Yalle| 30|         4| 34000|
| Hiba| 29|         4| 20000|
|Nouri| 24|         2| 15000|
| Paul| 21|         3| 20000|
+-----+---+----------+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['Name', 'age', 'Experience', 'Salary']

**[age, Experience] = features** 

In [19]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols = ["age","Experience"], outputCol = "Independent Features")

In [20]:
output = featureassembler.transform(training)

In [21]:
output.show()

+-----+---+----------+------+--------------------+
| Name|age|Experience|Salary|Independent Features|
+-----+---+----------+------+--------------------+
| Omar| 31|         3| 30000|          [31.0,3.0]|
| Hamm| 39|        11|120000|         [39.0,11.0]|
|Yalle| 30|         4| 34000|          [30.0,4.0]|
| Hiba| 29|         4| 20000|          [29.0,4.0]|
|Nouri| 24|         2| 15000|          [24.0,2.0]|
| Paul| 21|         3| 20000|          [21.0,3.0]|
+-----+---+----------+------+--------------------+



In [22]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [24]:
finalized_data = output.select('Independent Features', "Salary")


In [25]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [31.0,3.0]| 30000|
|         [39.0,11.0]|120000|
|          [30.0,4.0]| 34000|
|          [29.0,4.0]| 20000|
|          [24.0,2.0]| 15000|
|          [21.0,3.0]| 20000|
+--------------------+------+



In [32]:
from pyspark.ml.regression import LinearRegression
train_data, test_data =  finalized_data.randomSplit([0.75,0.25])

regression = LinearRegression(featuresCol = 'Independent Features', labelCol= "Salary")

regressor = regression.fit(train_data)

22/11/19 23:05:15 WARN Instrumentation: [6964ee20] regParam is zero, which might cause numerical instability and overfitting.


In [33]:
regressor.coefficients

DenseVector([779.267, 10556.0722])

In [34]:
pred_results = regressor.evaluate(test_data)

In [35]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [29.0,4.0]| 20000|37466.35667396057|
+--------------------+------+-----------------+

